<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# <font color="#76b900">**노트북 7: LangChain, RAG, 그리고 Memory를 활용한 지능형 대화시스템 설계**</font>

이번에는 LLM 기반 애플리케이션을 보다 지능적이고 문맥에 맞게 설계하기 위한 핵심 기술들을 다룹니다.  
실제 환경에서 더욱 실용적인 LLM 기반 서비스—예컨대 **문서 기반 QA 시스템**, **개인화 챗봇**, **지속 대화 기반 상담 시스템**—을 구성하기 위한 핵심 기술들을 단계적으로 익히게 됩니다.

- **LangChain**은 다양한 구성 요소를 체이닝(chain)하여 복잡한 LLM 파이프라인을 간단하고 유연하게 구성할 수 있도록 도와주는 프레임워크입니다.  
- **RAG(Retrieval-Augmented Generation)**는 LLM이 외부 지식 기반(예: 문서, DB 등)에 접근하여 보다 사실적이고 정보 기반의 응답을 생성할 수 있게 해주는 기술입니다.  
- **Memory**는 사용자의 이전 발화를 기억하고 이를 대화 흐름에 반영함으로써, 지속적이고 문맥에 맞는 대화를 가능하게 합니다.


LangChain의 구조적 접근과 RAG의 정보 강화 능력, 그리고 Memory를 활용한 상태 지속 대화는, 오늘날 고급 챗봇이나 에이전트 시스템에서 매우 중요한 구성 요소입니다.


### **학습 목표:**

이번 학습을 마치면 다음과 같은 내용을 습득하게 됩니다:

- LangChain 프레임워크의 기본 구성 요소 이해 및 활용  
- RAG (검색 기반 생성) 구조와 동작 방식 파악  
- Embedding, Vector Store, Retriever 등의 구성 요소 연결 실습  
- 대화 내용을 기억하고 맥락을 유지하는 Memory 체계 도입 방법 학습  
- LangChain Runnable 및 LCEL 스타일을 활용한 체인 구성 실습  
- 실제 문서 기반 QA 또는 상담형 챗봇 설계를 위한 기초 구축


### LangChain이란 무엇인가요?

LLM을 직접 사용할 수도 있지만, 실제 응용 서비스(예: 챗봇, 검색기반 응답, 상담 시스템 등)를 만들기 위해서는 다음과 같은 기능이 필요합니다:

- 대화 흐름 제어 (프롬프트 조립, 조건 분기)
- 외부 데이터 접근 (문서, DB, 검색 등)
- 사용자 맞춤형 처리 (기억, 대화 이력)
- 도구와 상호작용 (계산기, 검색 API, 코드 실행 등)

이런 기능들을 직접 구현하려면 많은 양의 코드가 필요합니다.  
**LangChain**은 이러한 복잡한 구조를 간단하게 구성할 수 있게 도와주는 **LLM 기반 애플리케이션 개발 프레임워크**입니다.

LangChain은 LLM을 중심으로 다음과 같은 구성 요소를 제공합니다:

- **Prompt Template**: 프롬프트를 동적으로 생성하는 템플릿
- **LLM Wrapper**: 다양한 모델(API, 로컬 모델 등)을 통일된 방식으로 호출
- **Chain**: 여러 단계의 처리를 연결해 하나의 흐름으로 만듦
- **Memory**: 이전 대화 내용을 기억하고 맥락 유지
- **Tool/Agent**: 도구를 통해 외부 시스템과 상호작용하거나 스스로 판단하여 행동




###  RAG (검색 기반 생성)이란?

기존의 LLM은 학습 당시의 정보만 알고 있습니다.  
**새로운 정보나 최신 데이터가 반영되지 않는다는 한계**가 있습니다.

예를 들어, 다음 질문을 생각해 봅시다:

> "2024년 이후 한국 정부의 AI 정책 중 주요한 내용은?"

이런 질문은 모델의 파라미터 안에 들어있을 가능성이 낮고,  
**외부 문서나 리포트에 있는 정보를 검색해서 답을 생성**해야 합니다.

이때 필요한 기술이 바로 **RAG: Retrieval-Augmented Generation**입니다.

RAG의 핵심 구조는 다음과 같습니다:

1. 사용자의 질문을 **임베딩(embedding)**으로 변환
2. 벡터 데이터베이스에서 관련 문서를 **검색(retrieve)**
3. 검색된 문서를 프롬프트에 포함하여 LLM이 **생성(generate)**

즉, LLM은 외부 지식을 바탕으로 답변을 "강화"해서 생성하게 됩니다.

### 실습을 위한 환경 안내

이 실습에서는 **NVIDIA의 NIM (NVIDIA Inference Microservices)** 기반 LLM을 사용합니다.  
모델은 사전 배포된 REST API 형태로 제공되며, 다음 코드를 통해 쉽게 연결할 수 있습니다:

In [ ]:
import requests
from langchain_nvidia_ai_endpoints import ChatNVIDIA

# 모델 경로 설정
model_path = "http://nim:8000/v1" 
# 모델 이름 가져오기
model_name = requests.get(f"{model_path}/models").json().get("data", [{}])[0].get("id")

# 환경 변수 설정
%env NVIDIA_BASE_URL=$model_path
%env NVIDIA_DEFAULT_MODE=open

# 모델이 LLM 클라이언트를 사용하는 경우 (예: llama-3.1)
if "llm_client" in model_path:
    model_name = "meta/llama-3.1-70b-instruct"
    
# ChatNVIDIA 인스턴스 생성
llm = ChatNVIDIA(model=model_name, base_url=model_path, max_tokens=5000, temperature=0)


#### 간단한 프롬프트 테스트

In [ ]:
llm.invoke("한국의 대표적인 AI 기업은 어디인가요?")
llm.invoke("삼성전자와 SK하이닉스의 차이점을 알려줘.")

In [ ]:
# temperature=0.0 → 가장 보수적이고 결정적인(deterministic) 응답
llm_0 = ChatNVIDIA(model=model_name, base_url=model_path, temperature=0.0)
# temperature=0.9 → 창의적이고 다양성이 높은 응답
llm_09 = ChatNVIDIA(model=model_name, base_url=model_path, temperature=0.9)

print("🔹 temperature=0.0:")
print(llm_0.invoke("인공지능이란 무엇인가요?"))

print("\n🔸 temperature=0.9:")
print(llm_09.invoke("인공지능이란 무엇인가요?"))

위 예제에서 LangChain 없이도 NVIDIA의 LLM을 직접 사용할 수 있음을 확인했습니다.  
이번에는 LangChain의 핵심 구성요소 중 하나인 **PromptTemplate**을 사용하여,  
프롬프트를 더 유연하고 재사용 가능하게 만드는 방법을 배워봅니다.

### 왜 PromptTemplate이 필요할까요?

이전 단계에서는 다음과 같이 직접 문장을 넣어서 LLM을 호출했습니다:

```python
llm.invoke("한국의 대표적인 AI 기업은 어디인가요?")
```
하지만 실제 서비스에서는 사용자 입력을 기반으로 동적으로 프롬프트를 구성해야 합니다.

예:
- 사용자가 입력한 질문을 그대로 반영
- 고정된 지시어(instruction)에 질문을 삽입 
- 여러 질문 형식을 통합적으로 처리

이러한 유연성을 위해 LangChain에서는 PromptTemplate 클래스를 제공합니다.

### PromptTemplate 사용 예제

In [ ]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 정의
prompt = PromptTemplate.from_template(
    "너는 한국어로 답변하는 유용한 AI 비서야. 다음 질문에 대답해줘:\n\n질문: {question}\n\n답변:"
)

# 템플릿에 값 채우기
filled_prompt = prompt.format(question="LangChain이 뭐야?")
print(filled_prompt)

### LLM과 PromptTemplate 연동

In [ ]:
# prompt.format() 결과를 llm에게 전달
response = llm.invoke(prompt.format(question="LangChain은 무엇이고, 어떤 상황에서 유용한가요?"))
print(response)


### 다양한 질문을 넣어 실험해보세요!

In [ ]:
questions = [
    "파이썬과 자바의 차이는?",
    "한국의 대표적인 반도체 기업은?",
    "GPT와 BERT의 차이점은?"
]

for q in questions:
    print(f" 질문: {q}")
    print(llm.invoke(prompt.format(question=q)))
    print("-" * 50)


### 정리
- PromptTemplate을 사용하면 질문 패턴을 일관되게 유지하면서 다양한 입력을 처리할 수 있습니다.
- 향후 RAG, Memory, Chain 등과 함께 사용할 때 매우 유용한 기본 구성입니다.

이번서는 **문서 기반 검색 시스템**의 핵심 구성 요소인 **벡터 데이터베이스(Vector DB)**를 구축해보겠습니다.  
이 과정을 통해 Retrieval-Augmented Generation(RAG)의 핵심 구조를 체험할 수 있습니다.


### Vector DB란?

Vector DB는 **문장을 수치 벡터로 변환(임베딩)**한 뒤,  
이 벡터들을 저장하고 **의미적으로 유사한 벡터를 빠르게 검색**할 수 있게 해주는 데이터베이스입니다.


### 전통적인 키워드 검색과 벡터 검색의 차이점

| 비교 항목 | 키워드 검색 | 벡터 검색 (Vector DB) |
|-----------|-------------|------------------------|
| 기준     | 단어 일치    | 의미 유사도             |
| 방식     | 전체 문자열에서 키워드 매칭 | 임베딩 벡터 간의 거리 기반 |
| 예시     | "AI 정책" → "AI 정책 보고서"만 검색 | "AI 전략", "인공지능 정책 방향"도 함께 검색 |
| 활용     | 전통적 검색엔진 | LLM 기반 RAG 시스템 등 |


### 장점과 단점

#### 장점
- **의미 기반 검색**: 단어가 일치하지 않아도 문맥이 비슷한 문서를 찾아줌
- **확장성**: 수천~수백만 개 문서도 유사도 기반으로 효율적 검색 가능
- **LLM 연계 최적화**: RAG, 에이전트 시스템 등에서 필수 기반

#### 단점
- **임베딩 코스트**: 문서를 미리 벡터로 변환해야 함 (시간+자원 소모)
- **정확도 보장 어려움**: 키워드 검색보다 정확한 매칭이 어려울 수 있음
- **저장 공간 필요**: 수치 벡터 저장이므로 일반 텍스트보다 용량 증가



### 실습 준비: 라이브러리 설치 (필요 시)

```bash
pip install langchain-community chromadb tiktoken


### 문서 나누기 + 임베딩 + 저장

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_nvidia_ai_endpoints import NvidiaEmbeddings
from langchain_community.vectorstores import Chroma

# 1. 문서 불러오기
loader = TextLoader("example.txt", encoding="utf-8")
docs = loader.load()

# 2. 문서 나누기 (chunk 단위로 자름)
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
splitted_docs = splitter.split_documents(docs)

# 3. NVIDIA 임베딩 모델 사용
embedding_model = NvidiaEmbeddings(model="nvidia/embeddings", base_url="http://nim:8000/v1")

# 4. ChromaDB에 저장
db = Chroma.from_documents(splitted_docs, embedding_model, persist_directory="./chroma_db")
db.persist()

### 저장된 벡터 확인 및 검색

In [ ]:
# 저장된 DB 로드
db = Chroma(persist_directory="./chroma_db", embedding_function=embedding_model)

# 질문 벡터화 후 검색
query = "한국의 AI 정책 방향은?"
results = db.similarity_search(query, k=3)

for i, doc in enumerate(results, 1):
    print(f"📄 유사 문서 {i}:")
    print(doc.page_content)
    print("-" * 50)


이번에는 지금까지 구축한 **문서 임베딩 + Vector DB**를 활용하여,  
**사용자의 질문에 가장 관련 있는 문서를 검색**한 뒤,  
**LLM이 해당 문서를 기반으로 답변을 생성하는 구조**, 즉 **RAG** 시스템을 직접 구현해봅니다.

### RAG 기본 구조 복습

사용자 질문
↓
[Retriever] → 관련 문서 검색
↓
[LLM] → 문서 기반 응답 생성

yaml
복사
편집

LangChain에서는 이 구조를 다음 구성 요소로 연결합니다:

- **VectorStoreRetriever**: 벡터 DB에서 유사 문서 검색
- **PromptTemplate**: 검색된 문서를 포함한 프롬프트 구성
- **LLMChain** or **Runnable**: LLM 호출

### Vector DB에서 Retriever 생성

In [ ]:
from langchain_community.vectorstores import Chroma

# 기존 저장된 Vector DB 로드
retriever = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embedding_model
).as_retriever()


### RAG용 프롬프트 구성

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """다음은 문서에서 검색된 내용입니다:

{context}

위 내용을 바탕으로 사용자의 질문에 답해주세요.

질문: {question}
답변:"""

prompt = PromptTemplate.from_template(template)


### Runnable로 체인 구성 (LCEL 방식)

In [ ]:
from langchain_core.runnables import RunnableMap, RunnablePassthrough

# Retriever 결과 → PromptTemplate에 연결 → LLM으로 전달
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

### 질문 → 답변 생성 실행

In [ ]:
response = rag_chain.invoke("한국의 인공지능 전략은 어떤 방향인가요?")
print(response)

이번에는 LangChain의 **Memory** 기능을 활용하여,  
사용자의 이전 질문과 대답을 기억하고, 대화의 흐름을 유지하는 방법을 배워봅니다.


### Memory가 왜 필요할까요?

기존 RAG 시스템은 **매 질문마다 독립적인 요청**으로 처리됩니다.

예:
```plaintext
Q: 한국의 AI 전략은 무엇인가요?  
A: 정부는 산업 기반 마련을 위해 여러 정책을 수립 중입니다.

Q: 그러면 의료 분야는요?  
→ [문맥 없음] → 질문이 무엇을 지칭하는지 불명확
```

### ConversationBufferMemory 구성

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 대화형 프롬프트 템플릿 구성
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 친절하고 유능한 한국어 AI 비서야."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

# 대화 이력 저장용 메모리 구성
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

### Memory + RAG + Runnable 연결

In [ ]:
from langchain_core.runnables import RunnableMap

# RAG 체인을 대화 흐름에 맞춰 재구성
conversational_rag = RunnableMap({
    "input": lambda x: x["question"],
    "chat_history": lambda _: memory.load_memory_variables({})["chat_history"]
}) | chat_prompt | llm

### 예시 대화

In [ ]:
# 첫 번째 질문
response1 = conversational_rag.invoke({"question": "한국의 인공지능 정책은 뭐야?"})
print(response1)
memory.save_context({"input": "한국의 인공지능 정책은 뭐야?"}, {"output": response1})

# 이어서 두 번째 질문
response2 = conversational_rag.invoke({"question": "의료 분야에는 어떻게 적용돼?"})
print(response2)
memory.save_context({"input": "의료 분야에는 어떻게 적용돼?"}, {"output": response2})


이제까지 학습한 요소들을 모두 통합하여 **문서 기반 검색 + 대화 기억 기능이 있는 지능형 대화 시스템**을 완성해보겠습니다.


### 전체 구조 요약
```
사용자 질문
↓
Memory → 이전 대화 기록 포함
↓
Retriever → 문서 검색
↓
LLM → 검색 문서 + 대화 문맥 바탕으로 답변 생성
```

### 통합 체인 구현 (Runnable 방식)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnableMap

# 프롬프트 템플릿 (검색된 문서 + 대화 흐름)
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 친절한 한국어 AI 비서야. 검색된 문서를 참고해서 질문에 정확히 답변해."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
    ("context", "{context}")
])

# 메모리 설정
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

# 통합 체인 구성
def get_chain(retriever, llm):
    return (
        RunnableMap({
            "context": lambda x: retriever.get_relevant_documents(x["question"]),
            "question": lambda x: x["question"],
            "chat_history": lambda _: memory.load_memory_variables({})["chat_history"]
        })
        | prompt
        | llm
    )

### 시스템 사용 예시

In [ ]:
# 체인 초기화
chain = get_chain(retriever, llm)

# 첫 질문
q1 = "대한민국의 인공지능 정책은 어떻게 되고 있어?"
a1 = chain.invoke({"question": q1})
print("답변1:", a1)
memory.save_context({"input": q1}, {"output": a1})

# 이어서 두 번째 질문
q2 = "의료 분야에는 어떻게 활용되고 있어?"
a2 = chain.invoke({"question": q2})
print("답변2:", a2)
memory.save_context({"input": q2}, {"output": a2})


# <font color="#76b900">**마무리**</font>

이번 단계에서는 지금까지 학습한 기능들을 통합하여 다음과 같은 목표를 달성했습니다:

- 문서 검색 기반의 RAG 시스템 구성  
- 대화 흐름을 기억하는 Memory 기능 적용  
- 사용자 질문에 대해 관련 문서를 기반으로 한 정확한 응답 생성  

이제 여러분은 LangChain과 NVIDIA LLM을 활용하여,  
**실제 서비스 수준의 문서 기반 QA 챗봇 시스템**을 구축할 수 있는 역량을 갖추게 되었습니다.

또한 추후 실제 서비스를 외부에 배포할 경우,  
**NVIDIA LLM 대신 OpenAI 등의 상용 API**를 연동하여 유연하게 확장할 수 있습니다.


In [ ]:
# ## 작업이 끝나면 실행해주세요!
# IPython 커널을 종료하기 위한 코드입니다
# IPython은 대화형 파이썬 실행 환경으로, 주피터 노트북 등에서 사용됩니다
import IPython                              # IPython 라이브러리를 불러옵니다
app = IPython.Application.instance()        # 현재 실행 중인 IPython 인스턴스를 가져옵니다
app.kernel.do_shutdown(True)                # IPython 커널을 안전하게 종료합니다 (True는 재시작하지 않음을 의미)


<center><a href="https://www.nvidia.com/ko-kr/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>